In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
from openspoor.mapservices import PUICMapservices, FeatureServerOverview
from openspoor.transformers import TransformerCoordinatesToSpoor, TransformerGeocodeToCoordinates, TransformerSpoortakToCoordinates
from openspoor.visualisations.trackmap import TrackMap, PlottingPoints, PlottingLineStrings, PlottingAreas, quick_plot
from openspoor.network.trackgraph import TrackNetherlands
from shapely.geometry import Point

featureserveroverview = FeatureServerOverview()

# Demo 1a) - Set up trackmap and add all ProRail gebieden, stations and geocodes in the Netherlands
Making a map consists of setting up a TrackMap object and adding the objects you wish to plot to it. These objects can be given in Pandas DataFrames, which are displayed on a map with added aerial photographs of the Dutch tracks and zoomed to the location of interest.
Optionally, these outputs can be saved as .html files which can then be shared or used in applications.
Within notebooks, TrackMap objects are displayed if their value is requested at the end of a cell.

Locations of many types of assets can be found on the publicly available ProRail mapservices, which can be queried in Python as per the below. These locations can then be displayed on the TrackMap.

For this demo we would like to plot the ProRail areas (gebied in Dutch), spoorhartlijnen and all stations in the Netherlands. As we don't know the exact names yet, or what data is being offered, we can first look for available datasets. Let's start with the areas.

In [3]:
featureserveroverview.search_for('gebied')

2025-07-22 08:53:04.854 | INFO     | openspoor.mapservices.find_mapservice:search_for:187 - Retrieving featureserver layers


<html lang="en">

<head>
<title>Folder: /</title>
<link href="/arcgis/rest/static/main.css" rel="stylesheet" type="text/css"/>
</head>

<body>

<table width="100%" class="userTable">
 <tr>
 <td class="titlecell">
 ArcGIS REST Services Directory
 </td>
 <td align="right">
   <a href="https://mapservices.prorail.nl/arcgis/rest/login">Login</a>
       
            | <a href="https://mapservices.prorail.nl/arcgis/tokens/">Get Token</a>
       </td>
 </tr>
</table>
<table width="100%" class="navTable">
<tr valign="top">
<td class="breadcrumbs">
<a href="/arcgis/rest/services">Home</a>

  &gt; <a href="/arcgis/rest/services">services</a>
  </td>
<td align="right">
<a href="https://mapservices.prorail.nl/arcgis/sdk/rest/02ss/02ss00000057000000.htm" target="_blank">Help</a> | <a href="https://mapservices.prorail.nl/arcgis/rest/services?f=help" target="_blank">API Reference</a>
</td>
</tr>
</table><table>
  <tr>
  <td class="apiref">
  <a href="?f=pjson" target="_blank">JSON</a>
     | <a href=

2025-07-22 08:53:20.237 | INFO     | openspoor.mapservices.find_mapservice:search_for:189 - Searching for "gebied"


,layer_url,description,server,version
31,https://mapservices.prorail.nl/arcgis/rest/ser...,Beperkingengebied spoor,Beperkingengebied,001
47,https://mapservices.prorail.nl/arcgis/rest/ser...,Saneringsgebied,Gebiedsindelingen,ProRail
48,https://mapservices.prorail.nl/arcgis/rest/ser...,Beperkingengebied,Gebiedsindelingen,ProRail
50,https://mapservices.prorail.nl/arcgis/rest/ser...,ProRail gebieden,Gebiedsindelingen,ProRail
72,https://mapservices.prorail.nl/arcgis/rest/ser...,Calamiteitenorganisatiegebied,Gebiedsindelingen,ProRail
73,https://mapservices.prorail.nl/arcgis/rest/ser...,Incidentbestrijdingsgebied,Gebiedsindelingen,ProRail
74,https://mapservices.prorail.nl/arcgis/rest/ser...,Incidentbestrijding vervangingsgebied,Gebiedsindelingen,ProRail
75,https://mapservices.prorail.nl/arcgis/rest/ser...,Bovenleiding Bedieningsgebied,Gebiedsindelingen,ProRail
76,https://mapservices.prorail.nl/arcgis/rest/ser...,Beperkingengebied spoor,Gebiedsindelingen,ProRail
77,https://mapservices.prorail.nl/arcgis/rest/ser...,Bediengebieden,Gebiedsindelingen,ProRail


Now that we know the full name of the service we want, we can query it directly. Similarly for the other 2 data sources.

In [ ]:
gebieden = featureserveroverview.search_for('ProRail gebieden', exact=True).load_data()
hartlijnen = featureserveroverview.search_for('Spoorbaanhartlijn (geocode)', exact=True).load_data()
stations = featureserveroverview.search_for('station', exact=True).load_data()

Now that we have gathered some data, we can try plotting it. As we haven't looked at what the data looks like, we can use the quick_plot function to get a quick idea of what is going on.

In [ ]:
quick_plot(hartlijnen, gebieden, stations)

This will convert the objects to plot into plottable objects. Three of these are currently supported, details can be found by using:
- ?PlottingPoints
- ?PlottingLineStrings
- ?PlottingAreas

We can therefore add some settings to the above plot to extend/improve the plot a bit:

In [ ]:
m = TrackMap([PlottingLineStrings(hartlijnen, color='SUBCODE', popup='GEOCODE', buffersize=100),
              PlottingAreas(gebieden, popup='NAAM', color='red'),
              PlottingPoints(stations, popup=['NAAM', 'STATIONSGROOTTE'], color_column='STATIONSGROOTTE')])
m.show(notebook=True)

# Demo 1b) - Customized markers
Markers can be customized to different colors, popup texts and looks.

In [ ]:
m=TrackMap()
# Add pretty markers with colors and custom Font Awesome icons at custom locations
m.add(PlottingPoints({'lat': [52.08, 52.093],
                      'lon': [5.119, 5.107],
                      'value': [1, 2],
                      "marker": ['train', "eye"]},
                     colors=('value', {(0, 1.5): 'blue', (1.5, 3.0): 'orange'}),
                     marker_column="marker"))

# Add clickable markers with arrows to indicate directions at locations
m.add(PlottingPoints({'lat': [52.0874117], 'lon': [5.1156098], "rotation": [245], 'location': ['ProRail Entrance']},
                     popup='location',
                     colors='purple',
                     rotation_column="rotation"))

# Plot clickable circle(s) on a map
m.add(PlottingPoints({'lat': [52.086420], 'lon': [5.113101], 'radius': [10], 'object': ['switch']},
                     popup='object',
                     markertype='circle',
                     radius_column="radius"))
m.show()

# Demo 2 - Obtain a track information based on X, Y coordinates
This demo uses the X,Y coordinates you have to find the track information of that point. It finds "spoortak", "geocode", "kilometrering" and "lokale kilometrering". Five coordinate examples cases are defined:
1. Case 1: A coordinate near a track. 
2. Case 2: A coordinate inside a switch. This is not currently supported.
3. Case 3: A coordinate on a crossing
4. Case 4: A coordinate near a crossing
5. Case 5: A coordinate outside the buffer distance of the tracks of 1.2 meters.

In [ ]:
# create dataframe of all the cases
xy_case_df = pd.DataFrame({'case_no': ["case_1", "case_2", "case_3", "case_4", "case_5"],
                           'x': [146506.901, 146970.582, 146445.417, 146465.756, 146406.901],
                           'y': [430192.467, 430102.380, 430101.289, 430102.479, 430192.467]})

# transform to a geopandas dataframe
xy_case_gdf = gpd.GeoDataFrame(xy_case_df,
                               geometry=gpd.points_from_xy(xy_case_df['x'], xy_case_df['y']),
                               crs="EPSG:28992")

# set the coordinate transformer
coordinates_transformer = TransformerCoordinatesToSpoor()

In [ ]:
# perform the transformation for our example cases
xy_extended_case_gdf = coordinates_transformer.transform(xy_case_gdf)

quick_plot(xy_extended_case_gdf.drop_duplicates(['case_no']),
               popup=['case_no', 'geocode_kilometrering', 'GEOCODE', 'SUBCODE'])

# Demo 3 - Obtain a X, Y coordinates based on track information

The second demo is the other way around. You already have a "spoortak" and its "lokale_kilometrering" and now we want to know what the X, Y Coordinate or GPS is.

In [ ]:
# create dataframe of all the cases
spoortak_case_df = pd.DataFrame(
    {'case_no': ["case_1", "case_2"],
     'SPOOR_ID': ['152_4123V_30.7', '152_4123V_30.7'],
     'lokale_kilometrering': [2, 18]
     }
)

In [ ]:
puic_mapservices = PUICMapservices()
spoortak_gdf = puic_mapservices.spoor_query.load_data()

In [ ]:
# set the spoortak transformer for 'Rijksdriehoek'
spoortak_transformer = TransformerSpoortakToCoordinates(
    'SPOOR_ID',
    'lokale_kilometrering',
    coordinate_system='Rijksdriehoek'  # 'GPS' if you want the GPS coordinates
)
spoortak_transformer = spoortak_transformer.fit(spoortak_gdf)

# perform the transformation for our example cases
spoortak_case_extended_df = spoortak_transformer.transform(spoortak_case_df)

# transform to a geopandas dataframe
spoortak_case_extended_gdf = gpd.GeoDataFrame(spoortak_case_extended_df,
                                              geometry=gpd.points_from_xy(spoortak_case_extended_df['x'],
                                                                          spoortak_case_extended_df['y']),
                                              crs="epsg:28992")

quick_plot(spoortak_case_extended_gdf, popup='case_no')

# Demo 4 -  Obtain a GPS coordinates based on track information

This time you already have a "geocode" and its "lokale_kilometrering" and we want to know what the X, Y Coordinate or GPS is. 
Two simple cases on the same "spoortak". You can see in the plot that X, Y coordinate is of the "spoorhartlijn". This can be explained by the fact that "geocode" and "geocode kilometrering" can have multiple "spoortakken".

In [ ]:
# create dataframe of all the cases
geocode_case_df = pd.DataFrame(
    {
        'case_no': ['case_1', 'case_2'],
        'Geocode': ['112', '009'],
        'geocode_km': [77, 115.208]
    }
)

In [ ]:
# set the geocode transformer for 'GPS'
geocode_transformer = TransformerGeocodeToCoordinates(
    geocode_column='Geocode',
    geocode_km_column='geocode_km',
    coordinate_system='GPS'  # 'Rijksdriehoek' if you want the GPS coordinates
)

# perform the transformation for our example cases
geocode_case_extended_df = geocode_transformer.transform(geocode_case_df)
quick_plot(geocode_case_extended_df, lat_column='x', lon_column='y', popup=['case_no'])

# Demo 5 -  Obtain a valid route between two given GPS coordinates

For two given GPS coordinates, this package allows the user to find a route from A to B, using the tracks in the Netherlands.
This generates a (cached) file which can quickly reconstruct a graph of the tracks in the Netherlands.
When deciding on a route, the graph has knowledge of which tracks are part of the same switch; at these location a kering (change of direction) can either be allowed or not.

In the below, we explore which routes can be taken to get from the northernmost point in the Netherlands to the most southern point which is still contained within the ProRail geometries.

In [ ]:
track = TrackNetherlands(local_cache=False)

start = Point(6.831006, 53.458520) # Northernmost track of the Netherlands
end = Point(5.692435, 50.737554)  # Southernmost track of the Netherlands

path_no_keringen = track.dijkstra(start, end, keringen_allowed=False)
path_keringen = track.dijkstra(start, end, keringen_allowed=True)

TrackMap([path_keringen.set_color('green'),
           path_no_keringen.set_color('orange')]).show(notebook=True)